Практическое задание 4
КМБО-06-20, Бундуки Владислав, Вариант 2

Устанавливаем необходимые библиотеки:

In [ ]:
!pip install pandas
!pip install sklearn

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
data = pd.read_csv('aug_test.csv', index_col='city')

#исключим столбец с уникальными значениями: enrollee_id	
data = data[data.columns.drop(['enrollee_id'])]
data.head()

,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
city,,,,,,,,,,,
city_41,0.827,Male,Has relevent experience,Full time course,Graduate,STEM,9,<10,NaN,1,21
city_103,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,5,NaN,Pvt Ltd,1,98
city_21,0.624,Male,No relevent experience,no_enrollment,High School,NaN,<1,NaN,Pvt Ltd,never,15
city_13,0.827,Male,Has relevent experience,no_enrollment,Masters,STEM,11,10/49,Pvt Ltd,1,39
city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,10000+,Pvt Ltd,>4,72


Описание столбцов:
enrollee_id :  уникальный идентификатор кандидата

city: Код города

city_ development _index : индекс развития города (в масштабе)

gender: Пол кандидата

relevent_experience: Соответствующий опыт кандидата

enrolled_university: Тип зачисленных университетских курсов, если таковые имеются.

education_level: Уровень образования кандидата

major_discipline :Обучение основной дисциплине кандидата

experience: Кандидатский общий стаж в годах

company_size: Количество сотрудников в компании текущего работодателя

company_type : Тип текущего работодателя

lastnewjob: разница в годах между предыдущей работой и текущей работой

training_hours: завершенные часы обучения

target: 0 - Не ищу смены работы, 1 - Ищу смену работы

In [2]:
#Выводим информацию о датасете
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2129 entries, city_41 to city_102
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city_development_index  2129 non-null   float64
 1   gender                  1621 non-null   object 
 2   relevent_experience     2129 non-null   object 
 3   enrolled_university     2098 non-null   object 
 4   education_level         2077 non-null   object 
 5   major_discipline        1817 non-null   object 
 6   experience              2124 non-null   object 
 7   company_size            1507 non-null   object 
 8   company_type            1495 non-null   object 
 9   last_new_job            2089 non-null   object 
 10  training_hours          2129 non-null   int64  
dtypes: float64(1), int64(1), object(9)
memory usage: 199.6+ KB


In [3]:
#Посмотрим на статистики численных данных:
data.describe()

,city_development_index,training_hours
count,2129.000000,2129.000000
mean,0.824984,64.983091
std,0.125074,60.238660
min,0.448000,1.000000
25%,0.698000,23.000000
50%,0.903000,47.000000
75%,0.920000,86.000000
max,0.949000,334.000000


In [4]:
#Кодируем столбец из категориального признака
data['gender'] = np.where(data['gender'] == 'Male', 1, 0)
data['education_level'] = np.where(data['education_level'] == 'Masters', 0, 1)
data['relevent_experience'] = np.where(data['relevent_experience'] == 'No relevent experience',0,1)

In [5]:
data.gender.head()

city
city_41     1
city_103    0
city_21     1
city_13     1
city_103    1
Name: gender, dtype: int32

In [6]:
data['experience'].value_counts()

>20    383
5      163
3      154
4      145
6      130
2      128
7      116
9      113
10      96
11      86
8       82
<1      74
16      68
15      59
1       56
14      55
13      54
12      52
17      36
19      29
18      26
20      19
Name: experience, dtype: int64

In [7]:
my_set = set(data.enrolled_university)
i = 0
for item in my_set:
    data['enrolled_university'] = data['enrolled_university'].replace(item, i)
    i = i + 1

    
my_set = set(data.last_new_job)
i = 0
for item in my_set:
    data['last_new_job'] = data['last_new_job'].replace(item, i)
    i = i + 1
    
my_set = set(data.major_discipline)
i = 0
for item in my_set:
    data['major_discipline'] = data['major_discipline'].replace(item, i)
    i = i + 1
    
my_set = set(data.company_type)
i = 0
for item in my_set:
    data['company_type'] = data['company_type'].replace(item, i)
    i = i + 1
    
my_set = set(data.company_size)
i = 0
for item in my_set:
    data['company_size'] = data['company_size'].replace(item, i)
    i = i + 1
    
my_set = set(data.experience)
i = 0
for item in my_set:
    data['experience'] = data['experience'].replace(item, i)
    i = i + 1

data.loc[data.experience == 'NaN' , 'experience'] = 0

data.loc[data.last_new_job == 'never' , 'last_new_job'] = 0
data.loc[data.last_new_job == 'NaN' , 'last_new_job'] = 0
data.loc[data.last_new_job == '>4' , 'last_new_job'] = 5

data.loc[data.gender == 'NaN' , 'gender'] = 0

data.head()

,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
city,,,,,,,,,,,
city_41,0.827,1,1,2,1,4,14,3,0,2,21
city_103,0.920,0,1,1,1,4,17,0,2,2,98
city_21,0.624,1,0,1,1,0,3,0,2,3,15
city_13,0.827,1,1,1,0,4,9,4,2,2,39
city_103,0.920,1,1,1,1,4,1,5,2,4,72


In [8]:
data

,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
city,,,,,,,,,,,
city_41,0.827,1,1,2,1,4,14,3,0,2,21
city_103,0.920,0,1,1,1,4,17,0,2,2,98
city_21,0.624,1,0,1,1,0,3,0,2,3,15
city_13,0.827,1,1,1,0,4,9,4,2,2,39
city_103,0.920,1,1,1,1,4,1,5,2,4,72
...,...,...,...,...,...,...,...,...,...,...,...
city_103,0.920,1,0,1,1,6,2,0,5,5,15
city_136,0.897,1,1,1,0,4,6,0,0,1,30
city_100,0.887,1,0,1,1,0,5,0,2,3,18


Выделяем целевую переменную

In [9]:
target = data.education_level
train = data.drop('education_level', axis = 1) 

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2129 entries, city_41 to city_102
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city_development_index  2129 non-null   float64
 1   gender                  2129 non-null   int32  
 2   relevent_experience     2129 non-null   int32  
 3   enrolled_university     2129 non-null   int64  
 4   major_discipline        2129 non-null   int64  
 5   experience              2129 non-null   int64  
 6   company_size            2129 non-null   int64  
 7   company_type            2129 non-null   int64  
 8   last_new_job            2129 non-null   int64  
 9   training_hours          2129 non-null   int64  
dtypes: float64(1), int32(2), int64(7)
memory usage: 166.3+ KB


Выделяем тренировочную и тестовую выборки

In [11]:
#y - целевая переменнная (target)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size = 0.3, random_state = 42)

N_train, _ = X_train.shape 
N_test,  _ = X_test.shape 
print (N_train, N_test)

1490 639


Построим классификатор логистической регрессии

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

grid={"C":np.logspace(-5,5,11), "penalty":["l1","l2"]}
logreg=LogisticRegression(solver='liblinear')
logreg_cv=GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(solver='liblinear'),
             param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05]),
                         'penalty': ['l1', 'l2']})

In [32]:
#проверка точности на тестовых данных
logreg_cv.cv_results_

{'mean_fit_time': array([0.00747979, 0.00807807, 0.00758743, 0.00917096, 0.00787861,
        0.01006553, 0.00797901, 0.01076093, 0.01516466, 0.0137713 ,
        0.03100114, 0.016854  , 0.03789721, 0.01795092, 0.0413918 ,
        0.01864216, 0.04098885, 0.01834991, 0.03949306, 0.01835465,
        0.04059699, 0.0183552 ]),
 'std_fit_time': array([0.00091921, 0.00029899, 0.00049503, 0.00132693, 0.00053869,
        0.00081854, 0.00063269, 0.00039341, 0.00115912, 0.00059313,
        0.00553769, 0.00053689, 0.0063233 , 0.00077242, 0.00545207,
        0.00133293, 0.00291408, 0.00101699, 0.00605344, 0.00091076,
        0.00285599, 0.00079562]),
 'mean_score_time': array([0.00438824, 0.00378995, 0.0043808 , 0.00409324, 0.00407872,
        0.00370421, 0.00379372, 0.00379453, 0.00358655, 0.00358999,
        0.00380104, 0.00359149, 0.00359175, 0.00359132, 0.003791  ,
        0.00389476, 0.00389071, 0.00379086, 0.00349183, 0.00378668,
        0.00378449, 0.00358119]),
 'std_score_time': array([0.00

In [31]:
logreg_cv.best_estimator_ 

LogisticRegression(C=0.1, penalty='l1', solver='liblinear')

In [116]:
#вывод подобранных параметров GreedSearchCV для каждого признака
answ = []
answ = logreg_cv.cv_results_['split9_test_score']

i = 0
for ind in data.columns:
    print(ind, "l1 =", answ[i], "l2 =", answ[i+1], sep = '\t')
    i += 2

city_development_index	l1 =	0.22818791946308725	l2 =	0.7718120805369127
gender	l1 =	0.7718120805369127	l2 =	0.7718120805369127
relevent_experience	l1 =	0.7718120805369127	l2 =	0.7718120805369127
enrolled_university	l1 =	0.7718120805369127	l2 =	0.7718120805369127
education_level	l1 =	0.7718120805369127	l2 =	0.7718120805369127
major_discipline	l1 =	0.7785234899328859	l2 =	0.7785234899328859
experience	l1 =	0.7718120805369127	l2 =	0.7718120805369127
company_size	l1 =	0.7718120805369127	l2 =	0.7718120805369127
company_type	l1 =	0.7718120805369127	l2 =	0.7718120805369127
last_new_job	l1 =	0.7718120805369127	l2 =	0.7718120805369127
training_hours	l1 =	0.7718120805369127	l2 =	0.7718120805369127


Оценим точность построенного классификатора с помощью метрик precision, recall и F1 на тестов

In [22]:
from sklearn.model_selection import cross_val_score
print("f1:"+str(np.average(cross_val_score(logreg_cv, X_test, y_test, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(logreg_cv, X_test, y_test, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(logreg_cv, X_test, y_test, scoring='recall'))))

f1:0.8586803331197951
precision:0.7535620547431571
recall:0.9979381443298969


Построим классификатор типа Случайный Лес (Random Forest) для решения той же задачи классификации

In [23]:
from sklearn.ensemble import RandomForestClassifier 
param_grid = { 'n_estimators': [50, 100, 150],'max_features': ['auto'],'max_depth' : list(range(1, 10)), 'criterion' :['gini']}
RandForCrit = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, cv = 5, refit = True) 
RandForCrit.fit(X_test, y_test)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'max_features': ['auto'],
                         'n_estimators': [50, 100, 150]})

Оценим точность построенного классификатора с помощью метрик precision, recall и F1

In [24]:
print("f1:"+str(np.average(cross_val_score(RandForCrit.best_estimator_, X_test, y_test, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(RandForCrit.best_estimator_, X_test, y_test, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(RandForCrit.best_estimator_, X_test, y_test, scoring='recall'))))

f1:0.8589274681471991
precision:0.7527436023622047
recall:1.0


Оценим точность построенного классификатора с помощью метрик precision, recall и F1 на тестовой выборке

In [112]:
from sklearn.model_selection import cross_val_score
print("f1:"+str(np.average(cross_val_score(logreg_cv, X_train, y_train, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(logreg_cv, X_train, y_train, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(logreg_cv, X_train, y_train, scoring='recall'))))

f1:0.8712088820250006
precision:0.7728470386188507
recall:0.9982608695652175


Построим классификатор типа Случайный Лес (Random Forest) для решения той же задачи классификации

In [43]:
from sklearn.ensemble import RandomForestClassifier 
param_grid = { 'n_estimators': [50, 100, 150],'max_features': ['auto'],'max_depth' : list(range(1, 10)), 'criterion' :['gini']}
RandForCrit = GridSearchCV(estimator = RandomForestClassifier(), param_grid = param_grid, cv = 5, refit = True) 
RandForCrit.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'criterion': ['gini'],
                         'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'max_features': ['auto'],
                         'n_estimators': [50, 100, 150]})

Оценим точность построенного классификатора с помощью метрик precision, recall и F1 на тестовой выборке

In [44]:
print("f1:"+str(np.average(cross_val_score(RandForCrit.best_estimator_, X_train, y_train, scoring='f1'))))
print("precision:"+str(np.average(cross_val_score(RandForCrit.best_estimator_, X_train, y_train, scoring='precision'))))
print("recall:"+str(np.average(cross_val_score(RandForCrit.best_estimator_, X_train, y_train, scoring='recall'))))

f1:0.8756736809001595
precision:0.7853222769793562
recall:0.9878524374176549


Сравним результаты:

f1 - лучше у классификатора Случайного леса

precision - лучше у классификатора Случайного леса

recall - лучше у классификатора Логистической регрессии